In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import copy
import os

In [2]:
def draw_pcd(file):
    """Visualize the point cloud of a file"""
    pcd = o3d.io.read_point_cloud(file, format = 'xyzrgb')
    o3d.visualization.draw_geometries([pcd])
    return pcd.points
#draw_pcd('1new.txt')

In [3]:
def cleanTxt(i):
    '''Takes in a txt file and with pandas saves a txt file that can be used by open3d'''
    data = pd.read_csv('/Users/lucadeste/Desktop/pc-im/'+str(i)+'.txt', header=None)
    np.savetxt(f'/Users/lucadeste/Desktop/FinalPointClouds/FinalPointClouds{i}', data.values)
    #i=0
#cleanTxt(f'{i}.txt',i)

In [4]:
def draw_registration_result(source, target, transformation):
    '''Draw the point cloud of two files'''
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])#Yellow
    target_temp.paint_uniform_color([0, 0.651, 0.929])#Cyan
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

def ICP(source,target):
    '''Algorithm of Iterative closest point that minimize the distance between two pointclouds'''
    threshold = 0.002
    trans_init = np.eye(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source=source, target=target, max_correspondence_distance=threshold, init=trans_init,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))

    print("Transformation is:")
    print(reg_p2p.transformation)
    print('Overlapping:',reg_p2p.fitness)
    #draw_registration_result(source, target, reg_p2p.transformation)
    return(reg_p2p)

In [5]:
def merge(source,target, transformationMatrix):
    '''Merge two pointcloud files into one pointcloud file'''
    pcd_combined = o3d.geometry.PointCloud()
    pcd_combined = source.transform(transformationMatrix)
    pcd_combined += target.transform(transformationMatrix)
    return pcd_combined


In [ ]:
def main():
    list_files1 = os.listdir('/Users/lucadeste/Desktop/pc-im')
    list_files1 = [l for l in list_files1 if l[-3::]=='txt']
    for i in range(len(list_files1)): #save each pointcloud into a better format (to be able to read it easily (with no comma))
        cleanTxt(i)
    list_files2 =  os.listdir('//Users/lucadeste/Desktop/FinalPointClouds')
    list_files2.remove('.DS_Store')
    print(list_files2)
    list_files2 = sorted(list_files2, key=lambda x: int(x[16:]))
    print('Modified point clouds:',list_files2)#I removed the first file (i don't know what is it but we don't need it)
    source = o3d.io.read_point_cloud('/Users/lucadeste/Desktop/FinalPointClouds/'+list_files2[0], format = 'xyz')
    for i in range(1,len(list_files2)):
        target = o3d.io.read_point_cloud('/Users/lucadeste/Desktop/FinalPointClouds/'+list_files2[i], format = 'xyz')
        reg_p2p = ICP(source,target)
        pcd_combined = merge(source,target, reg_p2p.transformation)
        source = pcd_combined
        o3d.io.write_point_cloud('Merging_file.ply',pcd_combined)
        merge_pcd = o3d.io.read_point_cloud('Merging_file.ply')
        print(len(np.asarray(merge_pcd.points)))
    o3d.visualization.draw_geometries([merge_pcd])
main()


['FinalPointClouds173', 'FinalPointClouds187', 'FinalPointClouds180', 'FinalPointClouds174', 'FinalPointClouds142', 'FinalPointClouds189', 'FinalPointClouds145', 'FinalPointClouds111', 'FinalPointClouds129', 'FinalPointClouds116', 'FinalPointClouds120', 'FinalPointClouds118', 'FinalPointClouds127', 'FinalPointClouds144', 'FinalPointClouds143', 'FinalPointClouds188', 'FinalPointClouds175', 'FinalPointClouds181', 'FinalPointClouds186', 'FinalPointClouds172', 'FinalPointClouds119', 'FinalPointClouds126', 'FinalPointClouds121', 'FinalPointClouds128', 'FinalPointClouds117', 'FinalPointClouds110', 'FinalPointClouds95', 'FinalPointClouds61', 'FinalPointClouds59', 'FinalPointClouds66', 'FinalPointClouds92', 'FinalPointClouds50', 'FinalPointClouds68', 'FinalPointClouds57', 'FinalPointClouds32', 'FinalPointClouds203', 'FinalPointClouds35', 'FinalPointClouds204', 'FinalPointClouds69', 'FinalPointClouds56', 'FinalPointClouds51', 'FinalPointClouds58', 'FinalPointClouds93', 'FinalPointClouds67', 'Fi

Transformation is:
[[ 1.00000000e+00 -3.30588980e-06 -5.69734142e-06 -6.93129230e-04]
 [ 3.30585556e-06  1.00000000e+00 -6.01034685e-06 -3.05911482e-04]
 [ 5.69736129e-06  6.01032801e-06  1.00000000e+00  6.07952954e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0103
50000
Transformation is:
[[ 1.00000000e+00 -4.41044320e-06  7.66835642e-06  9.19361950e-04]
 [ 4.41047275e-06  1.00000000e+00 -3.85324521e-06  6.73632956e-06]
 [-7.66833943e-06  3.85327903e-06  1.00000000e+00  2.87951859e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.00112
60000
Transformation is:
[[ 1.00000000e+00  7.88594164e-06 -2.03809218e-05 -1.49705983e-03]
 [-7.88587299e-06  1.00000000e+00  3.36857902e-06  2.18295509e-05]
 [ 2.03809484e-05 -3.36841830e-06  1.00000000e+00 -2.51463131e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.011866666666666666
70000
Transformation is:
[[ 9.99999999e-01  3.462

310000
Transformation is:
[[ 1.00000000e+00 -5.36318563e-06  2.51389325e-05  8.37891286e-04]
 [ 5.36337287e-06  1.00000000e+00 -7.44806233e-06 -4.15976174e-04]
 [-2.51388926e-05  7.44819715e-06  1.00000000e+00 -2.03617092e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.002667741935483871
320000
Transformation is:
[[ 1.00000000e+00 -2.68783251e-06  7.78188567e-06  5.05854699e-04]
 [ 2.68832578e-06  9.99999998e-01 -6.33876550e-05 -2.81877416e-03]
 [-7.78171528e-06  6.33876760e-05  9.99999998e-01  6.14479398e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.003540625
330000
Transformation is:
[[ 1.00000000e+00  1.48392855e-05 -1.89902702e-05 -7.61831805e-04]
 [-1.48382097e-05  9.99999998e-01  5.66505244e-05  2.62671649e-03]
 [ 1.89911109e-05 -5.66502426e-05  9.99999998e-01  6.31142646e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0006757575757575758
340000
Transformation

Transformation is:
[[ 1.00000000e+00  4.24283822e-06  3.02959020e-05  1.27718040e-03]
 [-4.23950399e-06  9.99999994e-01 -1.10054423e-04 -4.57747371e-03]
 [-3.02963688e-05  1.10054294e-04  9.99999993e-01 -6.74244003e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0012543859649122807
580000
Transformation is:
[[ 9.99999981e-01  1.63600466e-04  1.04669291e-04  4.00788932e-03]
 [-1.63597820e-04  9.99999986e-01 -2.52920996e-05  7.40592634e-04]
 [-1.04673428e-04  2.52749755e-05  9.99999994e-01  5.86453643e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0005724137931034483
590000
Transformation is:
[[ 9.99999975e-01  1.50733831e-04 -1.62358380e-04 -7.74574396e-03]
 [-1.50752650e-04  9.99999982e-01 -1.15905152e-04 -4.72926292e-03]
 [ 1.62340906e-04  1.15929625e-04  9.99999980e-01 -1.04837626e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.00036440677966101696
600000
Transform

Transformation is:
[[ 1.00000000e+00 -5.47604515e-06 -4.45888252e-06  2.68557379e-04]
 [ 5.47606347e-06  1.00000000e+00  4.10833920e-06 -5.43406309e-05]
 [ 4.45886002e-06 -4.10836362e-06  1.00000000e+00  4.54043449e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0010768292682926828
830000
Transformation is:
[[ 1.00000000e+00  1.13443276e-05 -1.52745989e-05 -9.24440237e-04]
 [-1.13443949e-05  1.00000000e+00 -4.40524906e-06 -8.58651805e-05]
 [ 1.52745490e-05  4.40542234e-06  1.00000000e+00 -1.51901626e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0007120481927710843
840000
Transformation is:
[[ 1.00000000e+00 -1.11214356e-05  1.54646549e-05  7.31318143e-04]
 [ 1.11214809e-05  1.00000000e+00 -2.92879351e-06  2.12250825e-04]
 [-1.54646223e-05  2.92896550e-06  1.00000000e+00 -1.48263038e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0010357142857142856
850000
Transforma

Transformation is:
[[ 1.00000000e+00 -1.07950152e-06 -2.31122674e-06 -8.62291280e-05]
 [ 1.07950870e-06  1.00000000e+00  3.10462655e-06  1.12587886e-04]
 [ 2.31122339e-06 -3.10462905e-06  1.00000000e+00  1.81701732e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.00029626168224299065
1080000
Transformation is:
[[ 1.00000000e+00  1.09726508e-05  6.45174181e-07 -6.44495025e-05]
 [-1.09726495e-05  1.00000000e+00 -2.01547649e-06  6.49708895e-05]
 [-6.45196296e-07  2.01546941e-06  1.00000000e+00 -2.75616280e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0014842592592592592
1090000
Transformation is:
[[ 1.00000000e+00 -7.76889276e-06  7.36521335e-06  2.03053331e-04]
 [ 7.76889863e-06  1.00000000e+00 -7.96717846e-07 -1.70774059e-04]
 [-7.36520716e-06  7.96775066e-07  1.00000000e+00  1.70167777e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0018752293577981652
1100000
Transf

Transformation is:
[[ 1.00000000e+00 -1.28731188e-06  1.26619587e-06 -5.75912363e-06]
 [ 1.28730858e-06  1.00000000e+00  2.60032122e-06  5.71964129e-07]
 [-1.26619921e-06 -2.60031959e-06  1.00000000e+00  2.18936707e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0022090909090909092
1330000
Transformation is:
[[ 1.00000000e+00 -5.99465896e-06  7.66594704e-07  1.21005723e-05]
 [ 5.99466164e-06  1.00000000e+00 -3.49684404e-06 -8.31546725e-05]
 [-7.66573741e-07  3.49684863e-06  1.00000000e+00 -4.94389665e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0022571428571428573
1340000
Transformation is:
[[ 1.00000000e+00  5.36210182e-06  1.60217200e-06  4.12036793e-05]
 [-5.36210482e-06  1.00000000e+00  1.87722727e-06  8.99659304e-05]
 [-1.60216193e-06 -1.87723586e-06  1.00000000e+00 -3.34984118e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0019940298507462686
1350000
Transfo

Transformation is:
[[ 1.00000000e+00 -3.81991785e-06 -1.12054324e-06  3.12474621e-05]
 [ 3.81991951e-06  1.00000000e+00  1.47983005e-06 -8.21273072e-06]
 [ 1.12053759e-06 -1.47983433e-06  1.00000000e+00 -2.09937712e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0020471337579617836
1580000
Transformation is:
[[ 1.00000000e+00  1.17672034e-06 -3.79083008e-06  6.90687603e-05]
 [-1.17672321e-06  1.00000000e+00 -7.57055758e-07 -6.37570161e-06]
 [ 3.79082919e-06  7.57060219e-07  1.00000000e+00 -3.82990891e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0013949367088607594
1590000
Transformation is:
[[ 1.00000000e+00  1.28990570e-06  1.63770223e-06 -6.74670841e-05]
 [-1.28990454e-06  1.00000000e+00 -7.04102448e-07  6.49721254e-06]
 [-1.63770314e-06  7.04100335e-07  1.00000000e+00 -2.07823759e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0024333333333333334
1600000
Transfo

1820000
Transformation is:
[[ 1.00000000e+00 -1.29601308e-07  7.82172898e-06 -2.67926664e-04]
 [ 1.29601181e-07  1.00000000e+00  1.62652148e-08  6.33352645e-07]
 [-7.82172898e-06 -1.62642011e-08  1.00000000e+00 -8.20879166e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0010615384615384616
1830000
Transformation is:
[[ 1.00000000e+00  3.24770965e-07 -1.09614280e-05  3.61218350e-04]
 [-3.24773601e-07  1.00000000e+00 -2.40494936e-07  3.35703398e-06]
 [ 1.09614279e-05  2.40498496e-07  1.00000000e+00  4.28054447e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.003155191256830601
1840000
Transformation is:
[[ 1.00000000e+00 -1.12306598e-06  5.57099076e-06 -1.58859071e-04]
 [ 1.12306430e-06  1.00000000e+00  3.01676407e-07 -6.64133592e-07]
 [-5.57099110e-06 -3.01670151e-07  1.00000000e+00 -9.10133331e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0023282608695652175
1850000


2070000
Transformation is:
[[ 1.00000000e+00 -5.24941666e-07 -9.47897313e-07 -3.46785716e-05]
 [ 5.24942594e-07  1.00000000e+00  9.78619727e-07 -4.43911274e-05]
 [ 9.47896799e-07 -9.78620225e-07  1.00000000e+00  1.06607732e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.003590338164251208
2080000
Transformation is:
[[ 1.00000000e+00 -1.06244602e-06 -2.30162658e-06 -3.37070870e-05]
 [ 1.06244458e-06  1.00000000e+00 -6.23364735e-07  4.08742101e-05]
 [ 2.30162724e-06  6.23362290e-07  1.00000000e+00  1.35569583e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0011408653846153847
2090000
Transformation is:
[[ 1.00000000e+00  9.06960710e-07  2.62664197e-06  1.79969643e-05]
 [-9.06961629e-07  1.00000000e+00  3.49821099e-07 -3.60095127e-05]
 [-2.62664166e-06 -3.49823481e-07  1.00000000e+00  4.28461685e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Overlapping: 0.0027751196172248805
2100000
